In [238]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)
library(ggplot2)
library(dummies)
library(stringr)
library(rms)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_score_finale.Rdata")
df <- df_score_finale
df <- subset(df,select= -c(degenzaPostTraOSP))

In [228]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips', 'Epatorenal', 
               'SpontanPeriton', 'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

In [157]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +39 secs: 

 confirmed 11 attributes: Albumin, Ascites, Centre, Duration, Lactates and 6 more;

 rejected 40 attributes: Age, Alcoholic, Anticoagulation, Cholestatic, cirrhosisDrugs and 35 more;

 still have 24 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +47 secs: 

 confirmed 3 attributes: Admsite, Norepinend, Openabdomen;

 rejected 6 attributes: Cavaveinanasth, Gender, Machi

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "PaO2FiO2"         "PlatCount"        "Albumin"          "Duration"        
 [9] "PRBCintraop"      "FFPintraop"       "Plateletsintraop" "CRIO"            
[13] "Crystalloids"     "PPS5"             "Norepinend"       "Lactates"        
[17] "ReperfusSy"       "Openabdomen"      "MELDNa"           "Nocomorbodities" 
[21] "Admsite"          "Encephalopathy"   "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [158]:
par(mar = c(13, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.3, las=2, xlab="", main="Variable Importance")

In [170]:
df_boruta$deg_cat <- df$deg_cat

In [171]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "PaO2FiO2"         "PlatCount"        "Albumin"          "Duration"        
 [9] "PRBCintraop"      "FFPintraop"       "Plateletsintraop" "CRIO"            
[13] "Crystalloids"     "PPS5"             "Norepinend"       "Lactates"        
[17] "ReperfusSy"       "Openabdomen"      "MELDNa"           "Nocomorbodities" 
[21] "Admsite"          "Encephalopathy"   "HCC"

In [229]:
df <- subset(df, select = c('MELDNa','PRBCintraop','Duration','Lactates','Albumin','deg_cat'))

In [230]:
df$deg_cat_bin = ifelse(df$deg_cat == 'fast',1,0)

In [231]:
head(df)

,MELDNa,PRBCintraop,Duration,Lactates,Albumin,deg_cat,deg_cat_bin
NA.575,10.26,5,347,2.3,4.0,fast,1
NA.577,26.68,44,361,2.4,3.0,NotFast,0
NA.682,4.44,0,261,2.8,4.3,fast,1
NA.687,25.78,14,370,2.5,2.7,NotFast,0
NA.692,4.14,0,314,1.7,4.5,fast,1
NA.693,7.29,1,317,1.6,4.5,fast,1


In [232]:
quantile(df)

ERROR: Error in `[.data.frame`(x, order(x, na.last = na.last, decreasing = decreasing)): undefined columns selected


In [233]:
df$dataset <- sample(x=c("train","validate"),
size=nrow(df),
replace=TRUE,
prob=c(0.75,0.25))

## LOESS smoothing curves for numeric variables

In [234]:
capOutlier <- function(x){
  qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
  caps <- quantile(x, probs=c(.05, .95), na.rm = T)
  H <- 1.5 * IQR(x, na.rm = T)
  x[x < (qnt[1] - H)] <- caps[1]
  x[x > (qnt[2] + H)] <- caps[2]
  return(x)
}

In [235]:
df[c('MELDNa','PRBCintraop','Duration','Lactates','Albumin')] <- lapply(df[c('MELDNa','PRBCintraop','Duration','Lactates','Albumin')], capOutlier)

In [236]:
nrow(df)

[1] 1148

In [210]:
df.cont<- df[df$dataset=="train",
c('deg_cat_bin','MELDNa','PRBCintraop','Duration','Lactates','Albumin')]

df.cat<- df[df$dataset=="train",
c('deg_cat')]




ymark<-seq(0,1,0.1)

for(var in names(df.cont)[-1]){
xvar<-seq(min(df.cont[,var]),
max(df.cont[,var]),
length.out=10)
    
    
mypath <- file.path("C:/Users/abonini/Documents/score",
paste(paste("plot",var,sep = "_"),
"pdf", sep = "."))

pdf(file=mypath)

gg<-ggplot(df.cont, aes(x=df.cont[,var],y=deg_cat_bin))+
geom_jitter(size=1, alpha=0.2,height=0.05)+
stat_smooth(method="loess",colour="blue", size=1.5)+
xlab(var)+
ylab("Probability on length of hospital stay")+
theme_bw()+
geom_hline(yintercept=ymark,col="red")+
scale_y_continuous(breaks=ymark)+
geom_vline(xintercept=xvar,col="green",alpha=0.5)+
scale_x_continuous(breaks=round(xvar,2))
    
print(gg)
dev.off()
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



## Choosing cut points and base reference

In [148]:
tmp <- df

In [154]:
MELDNacut<-c(-2.3,-1.7,0.18,0.8)
MELDNab<-"[-2.97,-2.3]"

Durationcat <-c(-3,-2.5,0.25,1.33)
Durationb<-"[-3.62,-3]"





cont.to.cat<-data.frame(id=c(1:nrow(df.cont)))

for (var in names(df.cont)[-1]) {
cat<-cut(df.cont[,var],
breaks=c(min(df.cont[,var]),
get(paste(var,"cut",sep="")),
max(df.cont[,var])),include.lowest= TRUE)
cat<-relevel(cat,ref=get(paste(var,"b",sep="")))
cont.to.cat<-cbind(cont.to.cat,cat)
}

ERROR: Error in get(paste(var, "cut", sep = "")): oggetto "PRBCintraopcut" non trovato


## Calculating scores for each level

In [ ]:
df.cont.to.cat<-cont.to.cat[,-1]

names(df.cont.to.cat)<-names(df.cont)[-1]

df.final<-cbind(cbind(df.cat,df.cont.to.cat),

mort.y=df.cont$mort.y)

mod<-glm(mort.y~.,
df.final, family="binomial")
score<-round(coef(mod)[-1])


score.cat<-score[1:3]
score.cont<-score[4:length(score)]


## Calculating scores for numeric variables

In [ ]:
var.cont<-as.character(1)

for(var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.cont<-c(var.cont,var.red)
}

var.cont<-unique(var.cont)[-1]

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.low<-as.numeric(str_extract(var,'(?<=(\\(|\\[))[0-9]+\\.*[0-9]*(?=\\,)'))
var.upper<-as.numeric(str_extract(var,'(?<=\\,)[0-9]+\\.*[0-9]*(?=\\])'))
df[,paste(var.red,"points",sep=".")]<-ifelse(
df[,var.red]<=var.upper&df[,var.red]>=var.low,
score[var],df[,paste(var.red,"points",sep=".")])
}

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-ifelse(
is.na(df[,paste(var,"points",sep=".")]),
0,df[,paste(var,"points",sep=".")])
}


## Converting the score to probability of mortality and comparing it to observed number of deaths

In [ ]:
glmod<-glm(mort.y~score,
df[df$dataset=="train",],
family="binomial")

newx<-seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score))

prd<-predict(glmod
newdata=data.frame(score=newx),
type="response",
se.fit=T)

count<-as.matrix(table(cut(df[df$dataset=="train",]$score,
breaks=seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score)),
include.lowest = T),
df[df$dataset=="train",]$mort.y))


## Visualization of the relationship between scores and probability of outcome events

In [ ]:
par(mar=c(5,4,4,5)+.1)

barplot(t(count),
main="Scores versus probability of death",
xlab="Scores",
ylab="Observed number of patients",
space=0,
col=c("yellow","lightblue"))

legend("topleft",fill=c("yellow","lightblue",NA),
lty = c(NA,NA,1),lwd=c(NA,NA,2),
legend=c("fast","NotFast",
"Predicted Prob."),
col=c("black"),
border = c("black","black",NA))

par(new=TRUE)

plot(prd$fit~newx,
type="l",col="black",
lwd=2,xaxt="n",yaxt="n",
xlab="",ylab="")

polygon(c(rev(newx), newx),
c(rev(prd$fit+1.96*prd$se.fit),
prd$fit-1.96*prd$se.fit),
col = adjustcolor('grey80',alpha=0.5),
border = NA)

lines(newx, prd$fit+1.96*prd$se.fit,
lty = 'dashed', col = 'red')

lines(newx, prd$fit-1.96*prd$se.fit,
lty = 'dashed', col = 'red')

axis(4)

mtext("Predicted probability of death",
side=4,line=3)


## Validation of the score

In [ ]:
ddist <- datadist(df)

options(datadist='ddist')

f.score<-lrm(mort.y~score,
df[df$dataset=="train",],
x=TRUE,y=TRUE)

phat.score<-predict(f.score,
df[df$dataset=="validate",],
type="fitted")

v.score<-val.prob(phat.score,
df[df$dataset=="validate",]$mort.y,
m=20)
